In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score

In [2]:
# 读取数据
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.head(10)

,online_order,book_table,rate,dish_liked,approx_cost_for_one_people,menu_item,original_index,cuisines_word_embedding_PC1,cuisines_word_embedding_PC2,cuisines_word_embedding_PC3,...,Marathahalli,New BEL Road,Old Airport Road,Rajajinagar,Residency Road,Sarjapur Road,Whitefield,predicted_rate,new_rate,label
0,0,0,3.8,2,300.0,0,4,-0.850827,-2.705486,0.586016,...,0,0,0,0,0,0,0,3.800000,3.8,0.52
1,1,0,4.2,7,250.0,1,34,-1.705105,-3.642225,1.086089,...,0,0,0,0,0,0,0,4.200000,4.2,0.56
2,1,0,3.8,7,100.0,0,40,-1.179024,-0.740344,-1.917100,...,0,0,0,0,0,0,0,3.800000,3.8,0.86
3,1,0,3.8,1,75.0,0,62,4.291739,-0.028077,0.780276,...,0,0,0,0,0,0,0,3.800000,3.8,0.60
4,1,0,3.8,1,250.0,0,80,-0.280420,-1.395033,0.889971,...,0,0,0,0,0,0,0,3.800000,3.8,0.68
5,0,0,3.8,2,200.0,0,103,-2.293814,1.254470,0.589672,...,0,0,0,0,0,0,0,3.800000,3.8,0.80
6,0,0,3.2,1,150.0,0,109,-1.037659,-0.791696,-1.929337,...,0,0,0,0,0,0,0,3.719161,3.4,0.66
7,0,0,0.0,1,150.0,0,114,-1.933474,1.686435,0.045060,...,0,0,0,0,0,0,0,3.582741,1.4,0.64
8,0,0,0.0,1,75.0,0,128,-0.648390,1.370996,-1.025303,...,0,0,0,0,0,0,0,3.174222,1.3,0.40
9,0,0,0.0,1,75.0,0,144,-1.456105,2.215195,1.329901,...,0,0,0,0,0,0,0,3.562133,1.4,0.60


In [4]:
test.head(10)

,online_order,book_table,rate,dish_liked,approx_cost_for_one_people,menu_item,original_index,cuisines_word_embedding_PC1,cuisines_word_embedding_PC2,cuisines_word_embedding_PC3,...,Malleshwaram,Marathahalli,New BEL Road,Old Airport Road,Rajajinagar,Residency Road,Sarjapur Road,Whitefield,predicted_rate,new_rate
0,1,1,4.1,7,400.0,0,0,1.372654,-1.112331,0.075874,...,0,0,0,0,0,0,0,0,4.10000,4.1
1,1,0,4.1,7,400.0,0,1,3.158016,-0.282180,0.549734,...,0,0,0,0,0,0,0,0,4.10000,4.1
2,1,0,3.8,7,400.0,0,2,0.368333,1.603899,-0.567267,...,0,0,0,0,0,0,0,0,3.80000,3.8
3,0,0,3.7,1,150.0,0,3,-1.179024,-0.740344,-1.917100,...,0,0,0,0,0,0,0,0,3.70000,3.7
4,1,0,3.8,7,300.0,0,5,-1.179024,-0.740344,-1.917100,...,0,0,0,0,0,0,0,0,3.80000,3.8
5,0,0,3.6,1,400.0,0,6,2.183805,-0.788710,0.186807,...,0,0,0,0,0,0,0,0,3.28963,3.5
6,1,1,4.6,7,300.0,0,7,-0.906553,1.744607,0.068235,...,0,0,0,0,0,0,0,0,4.60000,4.6
7,1,0,4.0,7,350.0,0,8,-0.104029,1.836365,-1.136420,...,0,0,0,0,0,0,0,0,4.00000,4.0
8,1,0,4.2,7,275.0,0,9,-0.507681,0.810681,-0.196371,...,0,0,0,0,0,0,0,0,4.20000,4.2
9,1,0,4.1,7,250.0,0,10,-1.628237,1.835902,-0.057209,...,0,0,0,0,0,0,0,0,4.10000,4.1


In [5]:
# 选择所有特征列，排除 'index' 列
feature_columns = [col for col in train.columns if col != 'original_index' and col != 'label' and col != 'predicted_rate' and col != 'rate' ] 
# 获取特征和标签
X_train = train[feature_columns]  # 只包含特征列
y_train = train['label']          # 标签列
X_test = test[feature_columns]    # 测试集只包含特征列

# 检查特征列
print("选定的特征列:", feature_columns)
print(len(feature_columns))
# 标准化特征
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

选定的特征列: ['online_order', 'book_table', 'dish_liked', 'approx_cost_for_one_people', 'menu_item', 'cuisines_word_embedding_PC1', 'cuisines_word_embedding_PC2', 'cuisines_word_embedding_PC3', 'cuisines_word_embedding_PC4', 'cuisines_word_embedding_PC5', 'cuisines_word_embedding_PC6', 'cuisines_word_embedding_PC7', 'cuisines_word_embedding_PC8', 'cuisines_word_embedding_PC9', 'cuisines_word_embedding_PC10', 'Buffet', 'Cafes', 'Delivery', 'Desserts', 'Dine-out', 'Drinks & nightlife', 'Pubs and bars', 'multiple_types', 'Bakery', 'Bar', 'Beverage Shop', 'Cafe', 'Casual Dining', 'Delivery.1', 'Dessert Parlor', 'Dhaba', 'Fine Dining', 'Food Court', 'Food Truck', 'Kiosk', 'Lounge', 'Mess', 'Microbrewery', 'Pub', 'Quick Bites', 'Sweet Shop', 'Takeaway', 'BTM', 'Banashankari', 'Bannerghatta Road', 'Basavanagudi', 'Bellandur', 'Brigade Road', 'Brookefield', 'Church Street', 'Electronic City', 'Frazer Town', 'HSR', 'Indiranagar', 'JP Nagar', 'Jayanagar', 'Kalyan Nagar', 'Kammanahalli', 'Koramangala 

### 随机森林回归

In [8]:
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

In [9]:
# 1. 使用 R² 作为准确性评估
scores_r2 = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
print("R² scores:", scores_r2)
print("Average R² score:", scores_r2.mean())

# 2. 使用负均方误差 (MSE) 作为评估指标
# `cross_val_score` 默认返回负数作为 MSE，便于排序，取负值得到实际的 MSE
scores_mse = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
print("Mean Squared Error (MSE):", -scores_mse)
print("Average MSE:", -scores_mse.mean())

# 3. 计算均方根误差 (RMSE)
# 通过将 MSE 的结果开平方得到 RMSE
scores_rmse = np.sqrt(-scores_mse)
print("Root Mean Squared Error (RMSE):", scores_rmse)
print("Average RMSE:", scores_rmse.mean())

# 4. 使用平均绝对误差 (MAE) 作为评估指标
scores_mae = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_absolute_error')
print("Mean Absolute Error (MAE):", -scores_mae)
print("Average MAE:", -scores_mae.mean())

R² scores: [-0.02755869 -0.03627656 -0.02305821 -0.07234317 -0.01557424]
Average R² score: -0.034962173224805194
Mean Squared Error (MSE): [0.03567646 0.03756034 0.03712082 0.03762401 0.03700651]
Average MSE: 0.036997627459974154
Root Mean Squared Error (RMSE): [0.18888213 0.1938049  0.19266763 0.19396911 0.19237076]
Average RMSE: 0.19233890520192384
Mean Absolute Error (MAE): [0.15755046 0.15851166 0.15858693 0.15876153 0.15818539]
Average MAE: 0.15831919367022207


### K-Neighbors 回归

In [10]:
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [11]:
# 1. 使用 R² 作为准确性评估
scores_r2 = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
print("R² scores:", scores_r2)
print("Average R² score:", scores_r2.mean())

# 2. 使用负均方误差 (MSE) 作为评估指标
# `cross_val_score` 默认返回负数作为 MSE，便于排序，取负值得到实际的 MSE
scores_mse = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
print("Mean Squared Error (MSE):", -scores_mse)
print("Average MSE:", -scores_mse.mean())

# 3. 计算均方根误差 (RMSE)
# 通过将 MSE 的结果开平方得到 RMSE
scores_rmse = np.sqrt(-scores_mse)
print("Root Mean Squared Error (RMSE):", scores_rmse)
print("Average RMSE:", scores_rmse.mean())

# 4. 使用平均绝对误差 (MAE) 作为评估指标
scores_mae = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_absolute_error')
print("Mean Absolute Error (MAE):", -scores_mae)
print("Average MAE:", -scores_mae.mean())

R² scores: [-0.12809362 -0.14193806 -0.11986109 -0.12751372 -0.14705767]
Average R² score: -0.13289283278226685
Mean Squared Error (MSE): [0.03931565 0.04138896 0.04086963 0.03931372 0.04185598]
Average MSE: 0.040548788010736414
Root Mean Squared Error (RMSE): [0.19828174 0.20344277 0.20216238 0.19827688 0.20458735]
Average RMSE: 0.2013502230662077
Mean Absolute Error (MAE): [0.16253414 0.16864322 0.1671799  0.16429347 0.16828543]
Average MAE: 0.1661872293192872


### 神经网络

In [12]:
model = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [13]:
# 1. 使用 R² 作为准确性评估
scores_r2 = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
print("R² scores:", scores_r2)
print("Average R² score:", scores_r2.mean())

# 2. 使用负均方误差 (MSE) 作为评估指标
# `cross_val_score` 默认返回负数作为 MSE，便于排序，取负值得到实际的 MSE
scores_mse = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
print("Mean Squared Error (MSE):", -scores_mse)
print("Average MSE:", -scores_mse.mean())

# 3. 计算均方根误差 (RMSE)
# 通过将 MSE 的结果开平方得到 RMSE
scores_rmse = np.sqrt(-scores_mse)
print("Root Mean Squared Error (RMSE):", scores_rmse)
print("Average RMSE:", scores_rmse.mean())

# 4. 使用平均绝对误差 (MAE) 作为评估指标
scores_mae = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_absolute_error')
print("Mean Absolute Error (MAE):", -scores_mae)
print("Average MAE:", -scores_mae.mean())

R² scores: [-5.03411015 -2.88950572 -1.57507385 -2.47970422 -7.59049557]
Average R² score: -3.913777901830975
Mean Squared Error (MSE): [0.15339138 0.12956238 0.15266167 0.16048592 0.16223694]
Average MSE: 0.15166765945076277
Root Mean Squared Error (RMSE): [0.39165212 0.35994775 0.39071943 0.40060694 0.40278647]
Average RMSE: 0.3891425414284614
Mean Absolute Error (MAE): [0.35635271 0.27214022 0.26336938 0.2358662  0.42795782]
Average MAE: 0.31113726688941246


### 梯度提升树

In [6]:
model = GradientBoostingRegressor(n_estimators=20, learning_rate=0.1)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

In [9]:
# 1. 使用 R² 作为准确性评估
scores_r2 = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
print("R² scores:", scores_r2)
print("Average R² score:", scores_r2.mean())

# 2. 使用负均方误差 (MSE) 作为评估指标
# `cross_val_score` 默认返回负数作为 MSE，便于排序，取负值得到实际的 MSE
scores_mse = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
print("Mean Squared Error (MSE):", -scores_mse)
print("Average MSE:", -scores_mse.mean())

# 3. 计算均方根误差 (RMSE)
# 通过将 MSE 的结果开平方得到 RMSE
scores_rmse = np.sqrt(-scores_mse)
print("Root Mean Squared Error (RMSE):", scores_rmse)
print("Average RMSE:", scores_rmse.mean())

# 4. 使用平均绝对误差 (MAE) 作为评估指标
scores_mae = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_absolute_error')
print("Mean Absolute Error (MAE):", -scores_mae)
print("Average MAE:", -scores_mae.mean())

R² scores: [0.07495172 0.09519119 0.08720802 0.06495909 0.09183947]
Average R² score: 0.08282989666024958
Mean Squared Error (MSE): [0.03223924 0.03279433 0.03331259 0.03260265 0.03313866]
Average MSE: 0.03281749385150026
Root Mean Squared Error (RMSE): [0.17955289 0.18109205 0.18251736 0.18056204 0.18204027]
Average RMSE: 0.18115292283012266
Mean Absolute Error (MAE): [0.15170274 0.15358193 0.15277179 0.15116968 0.15152845]
Average MAE: 0.15215091918099063


### XGBoosting

In [15]:
# 创建 XGBoost 回归模型
model = xgb.XGBRegressor(
    n_estimators=30,         # 树的数量，增加可以提高拟合效果但会增加计算成本
    learning_rate=0.1,       # 学习率，通常设置较小值（如 0.01-0.1），较低的学习率更稳健
    max_depth=3,             # 树的深度，增加可以提高模型复杂度，但可能过拟合
    objective='reg:squarederror',  # 目标函数为平方误差，适用于回归
    random_state=42          # 随机种子以确保可重复性
)
# 训练模型
model.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=30, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [7]:
# 定义要搜索的参数网格
param_grid = {
    'n_estimators': [50, 100, 200],         # 尝试不同数量的树
    'learning_rate': [0.01, 0.1, 0.2],      # 不同的学习率
    'max_depth': [3, 5, 7],                 # 树的不同深度
    'subsample': [0.8, 1.0],                # 每棵树的样本采样率
    'colsample_bytree': [0.8, 1.0],         # 每棵树的特征采样率
    'gamma': [0, 0.1, 0.2]                  # 正则化参数
}

# 初始化 XGBoost 回归模型
model1 = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# 使用 GridSearchCV 进行超参数搜索
model = GridSearchCV(estimator=model1, param_grid=param_grid, 
                           scoring='r2', cv=5, n_jobs=-1, verbose=2)

# 进行训练并搜索最佳参数
model.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.1, 0.2],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'n_estimators': [50, 100, 200],
                         'subsample': [0.8, 1.0]},
             scoring='r2', verbose=2)

In [10]:
# 输出最佳参数组合和对应的 R² 分数
print("Best parameters found: ", model.best_params_)
print("Best R² score: ", model.best_score_)

Best parameters found:  {'colsample_bytree': 0.8, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
Best R² score:  0.08512755421636949


In [ ]:
# 1. 使用 R² 作为准确性评估
scores_r2 = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
print("R² scores:", scores_r2)
print("Average R² score:", scores_r2.mean())

# 2. 使用负均方误差 (MSE) 作为评估指标
# `cross_val_score` 默认返回负数作为 MSE，便于排序，取负值得到实际的 MSE
scores_mse = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
print("Mean Squared Error (MSE):", -scores_mse)
print("Average MSE:", -scores_mse.mean())

# 3. 计算均方根误差 (RMSE)
# 通过将 MSE 的结果开平方得到 RMSE
scores_rmse = np.sqrt(-scores_mse)
print("Root Mean Squared Error (RMSE):", scores_rmse)
print("Average RMSE:", scores_rmse.mean())

# 4. 使用平均绝对误差 (MAE) 作为评估指标
scores_mae = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_absolute_error')
print("Mean Absolute Error (MAE):", -scores_mae)
print("Average MAE:", -scores_mae.mean())

In [8]:
# 在测试集上预测
test['predicted_value'] = model.predict(X_test_scaled).round(2)

# 将预测结果保存到新的 CSV 文件
test.to_csv("CAR_GB.csv", index=False)
print("已将预测结果保存到 'CAR_GB.csv'")

已将预测结果保存到 'CAR_GB.csv'


In [9]:
result = pd.read_csv("CAR_GB.csv")
result.head(10)

,online_order,book_table,rate,dish_liked,approx_cost_for_one_people,menu_item,original_index,cuisines_word_embedding_PC1,cuisines_word_embedding_PC2,cuisines_word_embedding_PC3,...,Marathahalli,New BEL Road,Old Airport Road,Rajajinagar,Residency Road,Sarjapur Road,Whitefield,predicted_rate,new_rate,predicted_value
0,1,1,4.1,7,400.0,0,0,1.372654,-1.112331,0.075874,...,0,0,0,0,0,0,0,4.10000,4.1,0.58
1,1,0,4.1,7,400.0,0,1,3.158016,-0.282180,0.549734,...,0,0,0,0,0,0,0,4.10000,4.1,0.60
2,1,0,3.8,7,400.0,0,2,0.368333,1.603899,-0.567267,...,0,0,0,0,0,0,0,3.80000,3.8,0.55
3,0,0,3.7,1,150.0,0,3,-1.179024,-0.740344,-1.917100,...,0,0,0,0,0,0,0,3.70000,3.7,0.63
4,1,0,3.8,7,300.0,0,5,-1.179024,-0.740344,-1.917100,...,0,0,0,0,0,0,0,3.80000,3.8,0.60
5,0,0,3.6,1,400.0,0,6,2.183805,-0.788710,0.186807,...,0,0,0,0,0,0,0,3.28963,3.5,0.59
6,1,1,4.6,7,300.0,0,7,-0.906553,1.744607,0.068235,...,0,0,0,0,0,0,0,4.60000,4.6,0.63
7,1,0,4.0,7,350.0,0,8,-0.104029,1.836365,-1.136420,...,0,0,0,0,0,0,0,4.00000,4.0,0.57
8,1,0,4.2,7,275.0,0,9,-0.507681,0.810681,-0.196371,...,0,0,0,0,0,0,0,4.20000,4.2,0.60
9,1,0,4.1,7,250.0,0,10,-1.628237,1.835902,-0.057209,...,0,0,0,0,0,0,0,4.10000,4.1,0.57


In [10]:
train.head(10)

,online_order,book_table,rate,dish_liked,approx_cost_for_one_people,menu_item,original_index,cuisines_word_embedding_PC1,cuisines_word_embedding_PC2,cuisines_word_embedding_PC3,...,Marathahalli,New BEL Road,Old Airport Road,Rajajinagar,Residency Road,Sarjapur Road,Whitefield,predicted_rate,new_rate,label
0,0,0,3.8,2,300.0,0,4,-0.850827,-2.705486,0.586016,...,0,0,0,0,0,0,0,3.800000,3.8,0.52
1,1,0,4.2,7,250.0,1,34,-1.705105,-3.642225,1.086089,...,0,0,0,0,0,0,0,4.200000,4.2,0.56
2,1,0,3.8,7,100.0,0,40,-1.179024,-0.740344,-1.917100,...,0,0,0,0,0,0,0,3.800000,3.8,0.86
3,1,0,3.8,1,75.0,0,62,4.291739,-0.028077,0.780276,...,0,0,0,0,0,0,0,3.800000,3.8,0.60
4,1,0,3.8,1,250.0,0,80,-0.280420,-1.395033,0.889971,...,0,0,0,0,0,0,0,3.800000,3.8,0.68
5,0,0,3.8,2,200.0,0,103,-2.293814,1.254470,0.589672,...,0,0,0,0,0,0,0,3.800000,3.8,0.80
6,0,0,3.2,1,150.0,0,109,-1.037659,-0.791696,-1.929337,...,0,0,0,0,0,0,0,3.719161,3.4,0.66
7,0,0,0.0,1,150.0,0,114,-1.933474,1.686435,0.045060,...,0,0,0,0,0,0,0,3.582741,1.4,0.64
8,0,0,0.0,1,75.0,0,128,-0.648390,1.370996,-1.025303,...,0,0,0,0,0,0,0,3.174222,1.3,0.40
9,0,0,0.0,1,75.0,0,144,-1.456105,2.215195,1.329901,...,0,0,0,0,0,0,0,3.562133,1.4,0.60


In [11]:
train["predicted_value"] = train["label"]

In [12]:
train = train.drop(columns = ["label"])
train

,online_order,book_table,rate,dish_liked,approx_cost_for_one_people,menu_item,original_index,cuisines_word_embedding_PC1,cuisines_word_embedding_PC2,cuisines_word_embedding_PC3,...,Marathahalli,New BEL Road,Old Airport Road,Rajajinagar,Residency Road,Sarjapur Road,Whitefield,predicted_rate,new_rate,predicted_value
0,0,0,3.8,2,300.0,0,4,-0.850827,-2.705486,0.586016,...,0,0,0,0,0,0,0,3.800000,3.8,0.52
1,1,0,4.2,7,250.0,1,34,-1.705105,-3.642225,1.086089,...,0,0,0,0,0,0,0,4.200000,4.2,0.56
2,1,0,3.8,7,100.0,0,40,-1.179024,-0.740344,-1.917100,...,0,0,0,0,0,0,0,3.800000,3.8,0.86
3,1,0,3.8,1,75.0,0,62,4.291739,-0.028077,0.780276,...,0,0,0,0,0,0,0,3.800000,3.8,0.60
4,1,0,3.8,1,250.0,0,80,-0.280420,-1.395033,0.889971,...,0,0,0,0,0,0,0,3.800000,3.8,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4971,1,1,4.2,7,800.0,0,51695,-0.086819,0.939226,-1.375560,...,0,0,0,0,0,0,1,4.200000,4.2,0.48
4972,1,1,4.1,7,600.0,0,51699,2.180580,1.256957,-0.781939,...,0,0,0,0,0,0,1,4.100000,4.1,0.80
4973,0,0,3.7,1,600.0,0,51706,0.170551,1.449433,-2.130870,...,0,0,0,0,0,0,1,3.553556,3.6,0.46
4974,0,0,3.9,1,1000.0,0,51707,-1.179024,-0.740344,-1.917100,...,0,0,0,0,0,0,1,3.900000,3.9,0.44


In [13]:
df = pd.concat([train, result],axis = 0, ignore_index = True)

In [14]:
df

,online_order,book_table,rate,dish_liked,approx_cost_for_one_people,menu_item,original_index,cuisines_word_embedding_PC1,cuisines_word_embedding_PC2,cuisines_word_embedding_PC3,...,Marathahalli,New BEL Road,Old Airport Road,Rajajinagar,Residency Road,Sarjapur Road,Whitefield,predicted_rate,new_rate,predicted_value
0,0,0,3.8,2,300.0,0,4,-0.850827,-2.705486,0.586016,...,0,0,0,0,0,0,0,3.800000,3.8,0.52
1,1,0,4.2,7,250.0,1,34,-1.705105,-3.642225,1.086089,...,0,0,0,0,0,0,0,4.200000,4.2,0.56
2,1,0,3.8,7,100.0,0,40,-1.179024,-0.740344,-1.917100,...,0,0,0,0,0,0,0,3.800000,3.8,0.86
3,1,0,3.8,1,75.0,0,62,4.291739,-0.028077,0.780276,...,0,0,0,0,0,0,0,3.800000,3.8,0.60
4,1,0,3.8,1,250.0,0,80,-0.280420,-1.395033,0.889971,...,0,0,0,0,0,0,0,3.800000,3.8,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51485,0,0,2.5,2,400.0,0,51711,1.914130,-1.237527,0.171139,...,0,0,0,0,0,0,1,2.500000,2.5,0.48
51486,0,0,3.6,1,750.0,0,51712,0.170551,1.449433,-2.130870,...,0,0,0,0,0,0,1,3.778667,3.7,0.50
51487,0,0,0.0,1,1000.0,0,51714,-1.037659,-0.791696,-1.929337,...,0,0,0,0,0,0,1,3.798222,1.5,0.42
51488,0,1,4.3,3,1250.0,0,51715,-1.179024,-0.740344,-1.917100,...,0,0,0,0,0,0,1,4.300000,4.3,0.63


In [15]:
df.to_csv("CAR_estimate_GB.csv")